In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

#import plotnine
#from plotnine import *

In [21]:
# 경로 설정
data_path = '/Users/kyoungseo/lgupls_aistage/data/'

In [22]:
# 데이터 불러오기 
df = pd.read_csv(os.path.join(data_path, 'sample.csv'), encoding='utf-8')

In [23]:
df = df.drop(['Unnamed: 0'], axis = 1)
df.head()

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,total_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,46.0,46.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,104.0,105.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,76.0,76.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,67.0,68.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,90.0,90.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04


In [24]:
X = df.loc[:, ~df.columns.isin(['album_id'])] # FEATURE DATA
y = df['album_id'] # LABEL DATA

In [7]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [33]:
## 전진 선택법

selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(X) > 0:
    remainder = list(set(X) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        x = df[selected_variables+[col]]
        x = sm.add_constant(x)
        model = sm.OLS(y,x).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [36]:
## 후진 소거법

selected_variables = X ## 초기에는 모든 변수가 선택된 상태
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0

while len(selected_variables) > 0:
    x = sm.add_constant(df[selected_variables])
    p_vals = sm.OLS(y,x).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
    max_pval = p_vals.max() ## 최대 p-value
    if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
 
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

ValueError: Boolean array expected for the condition, not int64

In [39]:
## 단계별 선택법

selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0

while len(X) > 0:
    remainder = list(set(X) - set(selected_variables))
    pval = pd.Series(index=remainder) 
    ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        x = df[selected_variables+[col]]
        x = sm.add_constant(x)
        model = sm.OLS(y,x).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = df[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [30]:
%%time
for i in ['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
          'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'] : 
    enc_1 = (X.groupby(i).size()) / len(df)
    enc_1
    
    X['{}_encode'.format(i)] = X[i].apply(lambda x : enc_1[x])
    
#인코딩 전 변수는 제거 
X = X.drop(['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
                'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'],axis=1)

Wall time: 1min 36s


In [26]:
X['short_trailer'] = X['short_trailer'].apply(lambda x: 1 if x == 'Y' else 0)
X['sex'] = X['sex'].apply(lambda x: 1 if x == 'M' else 0)

In [27]:
X['ss_id'] = pd.to_numeric(pd.to_datetime(X['ss_id']))
X['log_time_x'] = pd.to_numeric(pd.to_datetime(X['log_time_x']))
X['log_time_y'] = pd.to_numeric(pd.to_datetime(X['log_time_y']))

In [28]:
X = X.drop(['diff', 'diff_ox', 'continuous_play_x', 'act_target_dtl_x', 'act_target_dtl'], axis = 1)

In [32]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1914994 entries, 0 to 1914993
Data columns (total 21 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   profile_id                       int64  
 1   ss_id                            int64  
 2   log_time_x                       int64  
 3   log_time_y                       int64  
 4   diff_ss                          float64
 5   watch_time                       float64
 6   total_time                       float64
 7   continuous_play_y                float64
 8   short_trailer                    int64  
 9   payment                          float64
 10  time_slot                        float64
 11  search_ox                        float64
 12  prefer                           float64
 13  sex                              int64  
 14  age                              int64  
 15  pr_interest_keyword_cd_1_encode  float64
 16  pr_interest_keyword_cd_2_encode  float64
 17  pr_inter